## Assignment - Part 1: Webscrapping and data transformation

### 1. Webscrapping

Import libraries required webscrapping and data transformation.

In [41]:
import requests
import lxml.html as lh
import pandas as pd

Scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, to obtain the data with postal codes

In [42]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)

Parse table header:

In [43]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name = t.text_content().rstrip() # removes \n from strings
    col.append((name, []))
#print out extracted headers    
print(col)

[('Postcode', []), ('Borough', []), ('Neighbourhood', [])]


Parse rest of the table rows:

In [44]:
# data is stored on the second row onwards
for j in range(1, len(tr_elements)):
    # T is our j'th row
    T = tr_elements[j]

    # Ensure that if row is not of size 3, the //tr data is not from postalcode table
    if len(T) != 3:
        break

    # i is the index of our column
    i = 0

    # Iterate through each element of the row
    for t in T.iterchildren():
        data = t.text_content().rstrip()
        # Append the data to the empty list of the i'th column
        col[i][1].append(data)
        # Increment i for the next column
        i += 1

In [45]:
x = [len(C) for (Postcode,C) in col]
print(x) # seems there are 288 rows

[288, 288, 288]


### 2. Data Transformation

Create data frame using the data scraped from website.

In [46]:
myDict={title:column for (title,column) in col}
df = pd.DataFrame(myDict)
df.columns= ["PostalCode", "Borough", "Neighborhood"]
print("Shape of dataframe: ", df.shape)
df.head()

Shape of dataframe:  (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Filter out boroughs that is Not assigned.

In [47]:
df2 = df[df.Borough != 'Not assigned']
print("Shape of dataframe after filtering: ", df2.shape)
df2.head()

Shape of dataframe after filtering:  (211, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If more than one neighborhood exist in one postal code area, then combine those rows into one row with the neighborhoods separated with a comma.

In [48]:
df_grouped = df2.groupby(['PostalCode','Borough'], group_keys=True)['Neighborhood'].apply(lambda tags: ','.join(tags)).to_frame()
df_grouped = df_grouped.reset_index()
print("Shape of dataframe after grouping: ", df_grouped.shape)
df_grouped.head()

Shape of dataframe after grouping:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then set the neighborhood to be the same as the borough.

In [49]:
#df_grouped.loc[df_grouped['Neighborhood'] == ('Not assigned'), 'Neighborhood'] = df_grouped['Borough'] #"Queen's Park"
df_grouped['Neighborhood'].loc[df_grouped['Neighborhood'] == ('Not assigned')]= df_grouped['Borough'] 
print("Shape of dataframe after adjusting non-assigned neighborhood: ", df_grouped.shape)
df_grouped.head(5)

Shape of dataframe after adjusting non-assigned neighborhood:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Print out number of rows and columns of the dataframe.

In [50]:
print("Number of rows = ", df_grouped.shape[0])
print("Number of columns = ", df_grouped.shape[1])

Number of rows =  103
Number of columns =  3


### 3. Test Cases

1. Check if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [56]:
df_grouped[df_grouped['Neighborhood'] == df_grouped['Borough']]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


2. Ensure there's no rows where Borough, Neighborhood columns have "Not assigned" values.

In [57]:
print("Number of rows where Borough column has 'Not assigned': ",df_grouped[df_grouped['Borough'] == "Not assigned"].shape[0])
print("Number of rows where Neighborhood column has 'Not assigned': ",df_grouped[df_grouped['Neighborhood'] == "Not assigned"].shape[0])

Number of rows where Borough column has 'Not assigned':  0
Number of rows where Neighborhood column has 'Not assigned':  0


3. Ensure there's no duplicated rows with PostalCode, because Neigborhood with same postal codes are grouped already

In [58]:
duplicateRowsDF = df_grouped[df_grouped.duplicated(['PostalCode'])] 
print("Duplicate Rows based on PostalCode column are:", duplicateRowsDF.shape[0])

Duplicate Rows based on PostalCode column are: 0


### 4. Export Dataframe CSV file to use in Part2

In [60]:
export_csv = df_grouped.to_csv (r'C:\Users\{myName}\OneDrive\Desktop\export_postalcode.csv', index = None, header=True)